# Introduction AMAD functions with exampes

This notebook shows introductional examples for AMAD support functions.

# Atmospheric Model

This first example shows the application of the Atmospheric model used for ISA and dISA condition.
This is an extension of ADRpy Atmosphere class with Bluesky speed conversions.

In [ ]:
from amad.tools.atmosBADA import AtmosphereAMAD
import amad.tools.unit_conversion as uconv
import numpy as np
import matplotlib.pyplot as plt
import itertools
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from itertools import cycle

In [ ]:
disa = 0.
atmos = AtmosphereAMAD(offset_deg=disa)
atmos2 = AtmosphereAMAD(offset_deg=40.)

alt = np.arange(-2000., 124000., 2000.)

# alt_m = alt / 3.28084
alt_m = uconv.ft2m(alt)
temp_C_isa = []
temp_C_disa = []
rho_isa = []
rho_disa = []
p_isa = []
p_disa = []
for elm in alt_m:
    temp_C_isa.append(uconv.k2degc(atmos.airtemp_k(alt=elm)))
    temp_C_disa.append(uconv.k2degc(atmos2.airtemp_k(alt=elm)))
    rho_isa.append(atmos.airdens_kgpm3(alt=elm))
    rho_disa.append(atmos2.airdens_kgpm3(alt=elm))
    p_isa.append(atmos.airpress_pa(alt=elm))
    p_disa.append(atmos2.airpress_pa(alt=elm))

In [ ]:
fig, axs =  plt.subplots(nrows=1, ncols=3, figsize=(24, 6))

axs[0].plot(temp_C_isa, alt, 'b', label='ISA')
axs[0].plot(temp_C_disa, alt, 'r', label='delta ISA+40')
axs[0].set(xlabel='temperatrure [DEG C]', ylabel='Altitude [ft]',
       title='ISA temperature BADA model')
axs[0].grid()
axs[0].legend()

axs[1].plot(rho_isa, alt, 'b', label='ISA')
axs[1].plot(rho_disa, alt, 'r', label='delta ISA+40')
axs[1].set(xlabel='Density [kg/m^3]', ylabel='Altitude [ft]',
       title='ISA Density BADA model')
axs[1].grid()
axs[1].legend()

axs[2].plot(p_isa, alt, 'b', label='ISA')
axs[2].plot(p_disa, alt, 'r', label='delta ISA+40')
axs[2].set(xlabel='pressure [Pa]', ylabel='Altitude [ft]',
       title='ISA Pressure BADA model')
axs[2].grid()
axs[2].legend()

# fig.savefig("ADRpy.png")
plt.show()


## Speed Conversion methods (Mach, CAS, TAS Speed table)
This example shows the application of the speed conversion calculation methods.

In [ ]:
dict_params = {
    'np_alt_ft': np.arange(0., 45000., 2000.),
    'np_CAS_kt': np.arange(150., 375., 25.),
    'np_Mach': np.arange(0.1, 1.0, 0.1),
    'np_disa': np.array([0.]),
    'np_tas_from_mach': np.array([0.]),
    'np_tas_from_cas': np.array([0.])
}

In [ ]:
def Params_DoE(axes: dict) -> pd.DataFrame:
    """Simple DoE from 1D samples in all axis directions"""
    return pd.DataFrame(list(itertools.product(*axes.values())),
    columns = list(axes.keys()))
    
df_doe = Params_DoE(dict_params)

In [ ]:
for index, row in df_doe.iterrows():
    alt_m_elm = uconv.ft2m(row[0])
    CAS_ms_elm = uconv.kt2ms(row[1])
    mach_elm = row[2]
    disa_elm =  row[3]
    atmos = AtmosphereAMAD(offset_deg=disa_elm)
    df_doe.at[index, 'np_tas_from_mach'] =  uconv.ms2kt(atmos.mach2tas(mach_elm, alt_m_elm))
    df_doe.at[index, 'np_tas_from_cas'] =  uconv.ms2kt(atmos.cas2tas(CAS_ms_elm, alt_m_elm))

In [ ]:
plt.figure()
#df_doe_Mach = df_doe[abs(df_doe['np_Mach'] - Mach) < 0.001]
list_color_cycle = cycle(mcolors.TABLEAU_COLORS)

for elm_mach in sorted(list(set(df_doe['np_Mach']))):
    color = next(list_color_cycle)
    df_doe_elm = df_doe[df_doe['np_Mach'] == elm_mach]
    plt.plot(df_doe_elm['np_tas_from_mach'], df_doe_elm['np_alt_ft'], color='blue', label='Mach='+str(elm_mach))
    text = '%.2f' %elm_mach
    id = df_doe_elm.last_valid_index()
    plt.text(df_doe_elm.at[id, 'np_tas_from_mach'], df_doe_elm.at[id, 'np_alt_ft'], text, fontsize=8,  color='blue')

for elm_cas in sorted(list(set(df_doe['np_CAS_kt']))):
    color = next(list_color_cycle)
    df_doe_elm = df_doe[df_doe['np_CAS_kt'] == elm_cas]
    plt.plot(df_doe_elm['np_tas_from_cas'], df_doe_elm['np_alt_ft'], color='red', label='CAS='+str(elm_cas))
    text = '%.0f' %elm_cas
    id = df_doe_elm.last_valid_index()
    plt.text(df_doe_elm.at[id, 'np_tas_from_cas']-10., df_doe_elm.at[id, 'np_alt_ft']-2000., text, fontsize=8,  color='red')

plt.xlabel('TAS [kt]')
plt.ylim(0., 50000.)
plt.grid()
plt.ylabel('Altitude [ft]')
str_title = "Speed Conversions CAS / Mach to TAS f(Altitude)"
plt.title(str_title)
# plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)

In [ ]:
# Speed of Sound in kt
uconv.ms2kt(340)

In [ ]:
# CAS of Mach 0.8 at FL330
uconv.ms2kt(atmos.mach2tas(0.8, uconv.ft2m(33000.)))

In [ ]:
# Cross Over Altitude for 270kt and MA 0.78
uconv.m2ft(atmos.crossoveralt(uconv.kt2ms(270.), 0.78)) 